In [1]:
import tensorflow as tf
import numpy as np 
import extract_dataset as dat
import time_series_const as time
import post_text_preprocess as pro
import tfidf as tfidf
import json
import pickle

###################################################################################
#                                                                                 #
#       DATASET EXTRACTION                                                        #
#                                                                                 #
###################################################################################
# 4664 labeled events 
# For the labels, the value is 1 if the event is a rumor, and is 0 otherwise. 
# The content of all the posts in are in json format (timestamp and text)
# where each file is named event_id.json, corresponding to individual event

n_ev = 4664 # number of evenements
#event_ids = np.zeros((n_ev,1),dtype=int) #events ids ---> works on mac but not on windows
event_ids = np.zeros((n_ev,1)) #events ids
#labels = np.zeros((n_ev,1),dtype=int) # labels 0 or 1 ---> works on mac but not on windows
labels = np.zeros((n_ev,1)) # labels 0 or 1

# Extract labels and corresponding event ids
event_related_posts = dat.extract_dataset(event_ids,labels,"Weibo.txt",n_ev)

# i=0 
# j=0
# for event_id in event_ids:
#     filename = 'Weibo/%d.json' %event_id #event file with corresponding posts
#     with open(filename, 'r') as myfile:
#         data=myfile.read()
#     myfile.close()
#     posts = json.loads(data) #event related posts
#     for post in posts:
#         event_related_posts[i,j]=post['t'] #timestamp of post
#         j=j+1
#     event_related_posts[i]= np.sort(event_related_posts[i])
#     i=i+1
#     j=0

#Split the data into training and testing sets 80/20
N_test = int(0.2 * event_related_posts.shape[0])

#event_related_posts_train = event_related_posts[N_test:event_related_posts.shape[0],:]
labels_train= labels[N_test:labels.shape[0],:]
event_ids_train = event_ids[N_test:event_ids.shape[0],:]

#event_related_posts_test = event_related_posts[0:N_test,:]
labels_test= labels[0:N_test,:]
event_ids_test = event_ids[0:N_test,:]

### load pre-saved arrays for time saving
#event_related_posts_train =np.load('event_related_posts_train_array.npy')
#event_related_posts_test =np.load('event_related_posts_test_array.npy')

###################################################################################
#                                                                                 #
#       EVENTS TIME SERIES CONSTRUCTION                                           #
#                                                                                 #
###################################################################################
N=10; # RNN reference length N
#time_series_train = time.events_time_series(event_related_posts_train,N)
#time_series_test = time.events_time_series(event_related_posts_test,N)
  
###################################################################################
#                                                                                 #
#       TEXTUAL FEATURES EXTRACTION FOR THE TIME SERIES EVENTS                    #
#                                                                                 #
################################################################################### 
# Each json file (name=event_id (=1st post ID)) contains posts texts (chinese)
# TF.IDF on each intervals => inputs to RNN
# read time series file and .json corresponding post texts  
    
rnn_data_train=[] #time series tfidf RRN input data for each event
rnn_data_test=[]

#tfidf event: List of event intervals. Each element is dict
#word:tfidf value. Intervals comprise several words-tfidf score pairs
#according to the time series construction of the posts
#we only keep tfidf scores

# for j in range(event_related_posts_train.shape[0]):
#     print("training event: %d/3732" %(j+1))
#     filename = 'Weibo/%d.json' %event_ids_train[j] #training event
#     with open(filename, 'r') as myfile:
#             data=myfile.read()
#     myfile.close()
#     posts = json.loads(data) #training event related posts
  
#     tfidf_event=[]
#     event = time_series_train[j]
#     for i in range(event.shape[0]): #TFIDF for each interval
#         temp = event[i]
#         post_text = pro.post_text_preprocess(temp,posts)
#         #TF-IDF
#         TF_IDF = tfidf.tfidf(post_text)
#         tfidf_event.append(TF_IDF)
#     rnn_data_train.append(tfidf_event)
#     posts=[]

# for j in range(event_related_posts_test.shape[0]):
#     print("test event: %d/932" %(j+1))
#     filename = 'Weibo/%d.json' %event_ids_test[j] #training event
#     with open(filename, 'r') as myfile:
#             data=myfile.read()
#     myfile.close()
#     posts = json.loads(data) #training event related posts
  
#     tfidf_event=[]
#     event = time_series_test[j]
#     for i in range(event.shape[0]): #TFIDF for each interval
#         temp = event[i]
#         post_text = pro.post_text_preprocess(temp,posts)
#         #TF-IDF
#         TF_IDF = tfidf.tfidf(post_text)
#         tfidf_event.append(TF_IDF)
#     rnn_data_test.append(tfidf_event)
#     posts=[]

# with open("training_event_time_series_tfidf.txt", "wb") as fp:
#       pickle.dump(rnn_data_train, fp)    
# with open("test_event_time_series_tfidf.txt", "wb") as fp:
#       pickle.dump(rnn_data_test, fp)


### load event time series txt files (to save time)
with open("training_event_time_series_tfidf.txt", "rb") as fp:   # Unpickling
          rnn_data_train=(pickle.load(fp))        
with open("test_event_time_series_tfidf.txt", "rb") as fp:   # Unpickling
          rnn_data_test=(pickle.load(fp))
          
          
# Keep only the K-most important score per interval
# pad the number of intervals (each event need same number of intervals)
# put each event in a numpy array







    



In [2]:
#Just checking what the max number of tf.idf values (maxK) inside any interval in the data is
maxK = 0
for event in rnn_data_train:
    maxK = max(len(max(event,key=len)),maxK)
print(maxK)

394388


In [3]:
k = 10 #the number of tf.idf values sorted in descending order we will keep for each interval

maxNrIntervals = max( len(max(rnn_data_train,key=len)), len(max(rnn_data_test,key=len)))
print(maxNrIntervals)
new_rnn_train = []
new_rnn_test = []
#Processing Training Data
for event in rnn_data_train:
    new_event = []
    for interval in event: 
        kInterval = sorted(interval, reverse=True)[:k]
        kInterval.extend([0]*(k-len(kInterval))) #append the interval with zeros until it has a length of k
        new_event.append(kInterval)
    while len(new_event) < maxNrIntervals:
        new_event.append([0]*k) #append the event with intervals of zeros until it has a length of maxNrIntervals
    new_rnn_train.append(new_event)

#Processing Test Data
for event in rnn_data_test:
    new_event = []
    for interval in event: 
        kInterval = sorted(interval, reverse=True)[:k]
        kInterval.extend([0]*(k-len(kInterval))) #append the interval with zeros until it has a length of k
        new_event.append(kInterval)
    while len(new_event) < maxNrIntervals:
        new_event.append([0]*k) #append the event with intervals of zeros until it has a length of maxNrIntervals
    new_rnn_test.append(new_event)
        
new_rnn_train = np.array(new_rnn_train) #convert the standard python lists to numpy arrays
new_rnn_test = np.array(new_rnn_test)
print(new_rnn_train.shape)
print(new_rnn_test.shape)

19
(3732, 19, 10)
(932, 19, 10)


In [4]:
tf.device('/gpu:1') #My best gpu is gpu:1, change to gpu:0 if you only have 1 gpu
CUDA_VISIBLE_DEVICES=1 

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

model = Sequential()
#model.add(LSTM(128, input_shape=(new_rnn_train.shape[1:]), activation='relu', return_sequences=True))
model.add(LSTM(128, input_shape=(new_rnn_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2)) #is this really necessary?

#model.add(LSTM(128, activation='relu'))
model.add(LSTM(128))
model.add(Dropout(0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(new_rnn_train,
          labels_train,
          epochs=3,
          validation_data=(new_rnn_test, labels_test))

Train on 3732 samples, validate on 932 samples
Epoch 1/3
3732/3732 [==============================] - 15s 4ms/sample - loss: 0.6707 - accuracy: 0.6160 - val_loss: 0.9874 - val_accuracy: 0.0762
Epoch 2/3
3732/3732 [==============================] - 1s 273us/sample - loss: 0.6366 - accuracy: 0.6417 - val_loss: 0.8779 - val_accuracy: 0.3830
Epoch 3/3
3732/3732 [==============================] - 1s 258us/sample - loss: 0.5901 - accuracy: 0.7010 - val_loss: 0.8176 - val_accuracy: 0.5429
